In [4]:
from Twitter import Data_crawl
from select import select
import pandas as pd
import tweepy
import requests
import datetime
import json
import matplotlib.pyplot as plt
from IPython.display import display
import time
import random
from threading import Thread
from tqdm import tqdm, trange
import multiprocessing as mp
from  datetime import datetime

In [13]:
now = datetime.now().strftime('%H:%M:%S')
print("Job " + str(0) + " finish at " + now)


Job 0 finish at 19:18:35


In [3]:
total = 174256 * 388444
tims = 0
progress = tqdm(total=total)

while tims < total:
    progress.update(1)
    tims += 1

51928680it [00:25, 2019104.02it/s]


KeyboardInterrupt: 

In [ ]:
today = '2022-10-08'  #filename = str(datetime.date.today()) # 用當天日期當作檔案名稱
filename = 'Biden/' + today + '/' + today
filename = filename + '_' + str(0) + '_retweeters.json'
filename

必要 parameters

In [ ]:
token = [
'AAAAAAAAAAAAAAAAAAAAACHvcgEAAAAA9lDgALcD%2FZRxz%2BFrzuc6B7IxJqc%3DnBTsObUCrmc50dlBlB81QMnqnab8oj5Oo7uy4uXMi7vVUOEJAu',
'AAAAAAAAAAAAAAAAAAAAAIPvbAEAAAAApt3kcfjyiVBOLVWMbi1Fu6CLL24%3DuIMwpFYplQqMd68VXXv4eU954tnOQu5vljRnV3xL292k4F5sFR',
'AAAAAAAAAAAAAAAAAAAAAOvxbAEAAAAAOu%2FTOVHN5ocKWaChbnjZwI%2FoGdw%3DC1laLqLeCuXiwjg1bkV7kEVMY3pd3kZHDA4TKeRjcR89kKQPeR',
'AAAAAAAAAAAAAAAAAAAAAC3ucgEAAAAANkLeJqeojqdGXtbsGknDrLiZuJ4%3DueRC8NneqPUawhC3rkI83DXgUk68poiPHc60YAMtKqDr9NFTHs',
'AAAAAAAAAAAAAAAAAAAAAHfucgEAAAAAWrhXRgMpo%2Fj7fHl9uqSiUlklY5M%3DQWmApmhHTWG0LcdB77fiYOTnB8mbpU4HcPLMeWl6mQmZr60mKZ',
'AAAAAAAAAAAAAAAAAAAAAJKnJgEAAAAAgpYwO66pBtdDUcwjHYnWbMY1ow8%3DJgCTSnmkqVE5M0ywnyoSvMX1Ywpz2ivYnONwQ0Aro7bgby6TUm',
'AAAAAAAAAAAAAAAAAAAAAJaOeQEAAAAAyvSZWQ6m9q4SaMT1t1WCH%2FOymj0%3DVgRvLEP51YERK403i2pk5JU05Lu0vXppgQHxvhhxcBIkftu1FH'
]

# 時間格式需要符合 twitter 的格式：Year-Month-date + "T" + Hour:Minute:Second + "Z"
end_time  = datetime.datetime.today() # end time is today's 00:00
end_time = end_time.strftime('%Y-%m-%dT')
end_time = end_time + "00:00:00Z"

start_time = datetime.datetime.today() - datetime.timedelta(days= 8) # start time is 7 days ago
start_time = start_time.strftime('%Y-%m-%dT')
start_time = start_time + "00:00:00Z"

In [ ]:
dataset = Data_crawl(token=token)
tweet_df = dataset.Get_Tweets_Dataframe(
    start_time=start_time,
    end_time=end_time
)

In [ ]:
dataset.Get_Retweeters_dict_MultiThread()

In [ ]:
class MyThread(Thread):
    def __init__(self, func, args):
        super(MyThread, self).__init__()
        self.func = func
        self.args = args
        self.result = 0
    def run(self):
        self.result = self.func(*self.args)

    def get_results(self):
        return self.result

In [ ]:
l = [1,2,3,4,5]
l = l[0:2]

In [ ]:
d = {1:2, 2:3, 4:5, 6:9, 3:0}
f = open("2022-10-107_retweeters.json", 'w') 
#t = json.load(f)
t = json.dump(d, f, indent=True)
f.close()

In [ ]:
def g(l:list):
    s = random.randint(3,6)
    print(s, " second")
    time.sleep(s)
    v = {s:l}
    #with open(str(s) + '.json', 'w') as f:
        #a = json.dumps(v)
        #f.write(a)
t = [1,2,3,4,5,6,7,8,9,10]
thread = []
r = []
for i in range(3):
    y = MyThread(func=g, args=(t,))
    thread.append(y)
for i in range(3):
    thread[i].start()
for i in range(3):
    thread[i].join()

r

In [ ]:
t1 = MyThread(func=g, args=(t,))
t2 = MyThread(func=g, args=(t,))
t1.start()
t2.start()
t1.join()
t2.join()
res1 = t1.get_results()
res2 = t2.get_results()
print(res1, res2)

In [ ]:
print(t)

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
a = [i for i in range(55)]
a = chunks(a, int(len(a)/3)+1)
len(list(a))